In [5]:
import gradio as gr
import requests
import time
from PIL import Image, ImageDraw
import random

def random_color():
    return(random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))



# prebuild-read 호출하는 메소드
def request_document_intelligence(file_path):
    endpoint = end_point
    headers = {
        "Ocp-Apim-Subscription-Key" : ocp_key,
        "Content-Type" : "image/png"

    }

    # file_path로부터 파일을 읽음(이미지) -> 바이너리로 불러오기 -> request body에 넣음
    with open(file_path, "rb") as image:
        image_data = image.read()
    
    response = requests.post(endpoint, headers=headers, data=image_data)
    #print(response.status_code, response.headers)

    if response.status_code == 202:
        result_url = response.headers['Operation-Location']

        while(True):
            result_response = requests.get(result_url, headers = {
                "Ocp-Apim-Subscription-Key" : headers["Ocp-Apim-Subscription-Key"]

            })

            result_response_json = result_response.json()
            result_status = result_response_json['status']

            if result_status == "running":
                print(result_status)
                time.sleep(1)
                continue

            else:
                break

        if result_status == "succeeded":
            print(result_status)
            print(result_response_json)
            return result_response_json
        else:
            return ""

        print(result_url)


    
    

    return ""




# polugon data + 입력이미지를 받아서 네모 박스를 그려주는 메소드
def draw_image(file_path, result_response):
    image = Image.open(file_path)
    draw = ImageDraw.Draw(image)

    # polygone 그려주기
    block_list = result_response['analyzeResult']['paragraphs']

    for block in block_list:
        line_color = random_color()
        polygon_list = block['boundingRegions'][0]['polygon']
        points = [(polygon_list[i], polygon_list[i+1]) for i in range(0, len(polygon_list), 2)] # 좌표를 튜플형으로 변환

        content = block['content']

        # point_list = list()
        # for i in range(0, len(polygon_list), 2):
        #     point_list.append((point_list[i], point_list[i+1]))
        
        draw.polygon(points, outline=line_color, width=2)
        draw.text((points[0][0], points[0][1] - 10), content, fill=line_color)
        #print(polygon_list)


    return image

with gr.Blocks() as demo:

    def click_send(file_path):
        result_response = request_document_intelligence(file_path)
        image = draw_image(file_path, result_response)
        return image

    input_image = gr.Image(label="입력 이미지", type="filepath")
    send_button = gr.Button("전송")
    output_image = gr.Image(label="출력 이미지", interactive=False, type = "pil")

    send_button.click(click_send, inputs = [input_image], outputs =[output_image])

demo.launch()
# file = "../document-intelligence/read-resume.png"
# result_response = request_document_intelligence(file)
# print(result_response)
# draw_image(file, result_response)

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


succeeded
{'status': 'succeeded', 'createdDateTime': '2025-03-19T08:55:34Z', 'lastUpdatedDateTime': '2025-03-19T08:55:34Z', 'analyzeResult': {'apiVersion': '2024-11-30', 'modelId': 'prebuilt-read', 'stringIndexType': 'textElements', 'content': 'Lorem ipsum dolor sit amet, consectetuer adipiscing elit, sed diam nonummy nibh euismod tincidunt ut laoreet dolore magna aliquam erat volutpat. Ut wisi enim ad minim veniam, quis nostrud exerci tation ullamcorper suscipit lobortis nisi ut aliquip ex ea commodo consequat. Duis autem vel eum iriure dolor in hendrerit in vulputate. Lorem ipsum dolor sit amet, consectetuer.\nEXPERIENCE\nLorem Ipsum 20XX-20YY\nLorem ipsum dolor sit amet, consectetuer adipiscing elit, sed diam nonummy nibh euismod tincidunt ut laoreet dolore magna aliquam erat volutpat. Ut wisi enim ad minim veniam, quis nostrud exerci tation ullamcorper suscipit lobortis nisi ut aliquip ex ea commodo consequat. Duis autem vel eum iriure dolor in hendrerit in vulputate.\nDolor Sit Am